### Data import

In [1]:
import pandas as pd

processed_data = pd.read_csv('data/occurences_processed_data.csv')
processed_data.head(3)

,Unnamed: 0,latitude,longitude,country,region,img_path
0,0,20.824885,-98.499517,Mexico,Hidalgo,0.png
1,1,-3.451752,-54.563937,Unknown,Unknown,1.png
2,2,-23.496464,-47.460542,Brazil,São Paulo,2.png


In [5]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

# Charger le jeu de données
file_path = "data/occurences_processed_data.csv"
data = pd.read_csv(file_path)

# Répertoire des images
image_dir = "1/dataset/"  # Remplacer par le chemin vers le dossier contenant les images

# Vérification de l'existence des fichiers d'image
data['img_path_full'] = data['img_path'].apply(lambda x: os.path.join(image_dir, x))
data = data[data['img_path_full'].apply(os.path.exists)]

# Encodage des labels (pays)
label_encoder = LabelEncoder()
data['country_encoded'] = label_encoder.fit_transform(data['country'])

# Split train/test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Préparation des générateurs d'images
img_size = (128, 128)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col='img_path_full',
    y_col='country_encoded',
    target_size=img_size,
    class_mode='raw',
    batch_size=batch_size
)

test_generator = test_datagen.flow_from_dataframe(
    test_data,
    x_col='img_path_full',
    y_col='country_encoded',
    target_size=img_size,
    class_mode='raw',
    batch_size=batch_size,
    shuffle=False
)

# Modèle CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entraînement
epochs = 10
history = model.fit(train_generator, validation_data=test_generator, epochs=epochs)

# Évaluation
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy:.2f}")


Found 7372 validated image filenames.
Found 1843 validated image filenames.


/Users/leo-paul/geoguess/geo-env/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/leo-paul/geoguess/geo-env/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 110s 473ms/step - accuracy: 0.2174 - loss: 2.9008 - val_accuracy: 0.3218 - val_loss: 2.4293
Epoch 2/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 112s 485ms/step - accuracy: 0.3297 - loss: 2.3544 - val_accuracy: 0.3467 - val_loss: 2.2772
Epoch 3/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 116s 500ms/step - accuracy: 0.3929 - loss: 2.0933 - val_accuracy: 0.3836 - val_loss: 2.0990
Epoch 4/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 114s 495ms/step - accuracy: 0.4457 - loss: 1.8704 - val_accuracy: 0.4069 - val_loss: 2.0565
Epoch 5/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 124s 535ms/step - accuracy: 0.4848 - loss: 1.7026 - val_accuracy: 0.4428 - val_loss: 1.9854
Epoch 6/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 113s 489ms/step - accuracy: 0.5669 - loss: 1.4016 - val_accuracy: 0.4650 - val_loss: 1.8982
Epoch 7/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 115s 498ms/step - accuracy: 0.6528 - loss: 1.1624 - val_accuracy: 0.4786 - val_loss: 2.0696
Epoch 8/10
231/231 ━━━━━━━━━━━━━━━━━━━━ 116s 501ms/step - accuracy: 0.7304 -

In [7]:
model.save("models/model_1shot.h5", save_format="tf")

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

def load_and_preprocess_image(img_path, img_size=(128, 128)):
    """Load an image and preprocess it for prediction."""
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Ajouter une dimension batch
    img_array /= 255.0  # Normalisation
    return img_array

def predict_country(img_path, model, label_encoder):
    """Predict the country from an image using the trained model."""
    img_array = load_and_preprocess_image(img_path)
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction)
    predicted_country = label_encoder.inverse_transform([predicted_class_index])[0]
    
    print(f"Predicted Country: {predicted_country}")
    return predicted_country

# Exemple d'utilisation :
# img_path_example = "1/dataset/260.png"  # Remplacer par le chemin de l'image de test
# predict_country(img_path_example, model, label_encoder)


In [8]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model


file_path = "data/occurences_processed_data.csv"
data = pd.read_csv(file_path)

# Répertoire des images
image_dir = "1/dataset/"  # Remplacer par le chemin vers le dossier contenant les images

# Vérification de l'existence des fichiers d'image
data['img_path_full'] = data['img_path'].apply(lambda x: os.path.join(image_dir, x))
data = data[data['img_path_full'].apply(os.path.exists)]

# Encodage des labels (pays)
label_encoder = LabelEncoder()
data['country_encoded'] = label_encoder.fit_transform(data['country'])

model = load_model("models/model_1shot.h5")

predict_country("sandbox_data/test_2.png", model, label_encoder)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Predicted Country: Australia


'Australia'

In [6]:
label_encoder.classes_

array(['Argentina', 'Australia', 'Bolivia', 'Botswana', 'Brazil',
       'Bulgaria', 'Cambodia', 'Colombia', 'Finland', 'Ghana', 'India',
       'Indonesia', 'Italy', 'Kenya', 'Mali', 'Mexico', 'Mongolia',
       'Nigeria', 'Peru', 'Poland', 'Romania', 'Russia', 'Senegal',
       'South Africa', 'Spain', 'Sweden', 'Thailand', 'Türkiye',
       'Ukraine', 'Unknown', 'Uruguay'], dtype=object)